# Prototype_1
### 사용자 화면에 출력하기

#### Requirements

In [ ]:
import os
import glob

import numpy as np
import pandas as pd
import json

import mediapipe as mp
import cv2
import pytube

#### Prototype

In [ ]:
class JustDDance:
    def __init__(self):
        self.dance_name = None
        self.download_path = None
        self.dance_cor = None
        self.user_cor = None
        self.pose_landmark_dict = {
            0: "nose",
            1: "left_eye_inner",
            2: "left_eye",
            3: "left_eye_outer",
            4: "right_eye_inner",
            5: "right_eye",
            6: "right_eye_outer",
            7: "left_ear",
            8: "right_ear",
            9: "left_mouth",
            10: "right_mouth",
            11: "left_shoulder",
            12: "right_shoulder",
            13: "left_elbow",
            14: "right_elbow",
            15: "left_wrist",
            16: "right_wrist",
            17: "left_pinky",
            18: "right_pinky",
            19: "left_index",
            20: "right_index",
            21: "left_thumb",
            22: "right_thumb",
            23: "left_hip",
            24: "right_hip",
            25: "left_knee",
            26: "right_knee",
            27: "left_ankle",
            28: "right_ankle",
            29: "left_heel",
            30: "right_heel",
            31: "left_foot_index",
            32: "right_foot_index"
        }
        self.hand_landmark_dict = {
                0: "wrist",
                1: "thumb_cmc",
                2: "thumb_mcp",
                3: "thumb_ip",
                4: "thumb_tip",
                5: "index_mcp",
                6: "index_pip",
                7: "index_dip",
                8: "index_tip",
                9: "middle_mcp",
                10: "middle_pip",
                11: "middle_dip",
                12: "middle_tip",
                13: "ring_mcp",
                14: "ring_pip",
                15: "ring_dip",
                16: "ring_tip",
                17: "pinky_mcp",
                18: "pinky_pip",
                19: "pinky_dip",
                20: "pinky_tip"
        }
    
    def download_video(self, download_path="./video/"):
        url = input("연습할 춤의 유튜브 링크: ")
        if not os.path.isdir(download_path): os.mkdir(download_path)
        yt = pytube.YouTube(url)
        stream = yt.streams.filter(res="720p").first()
        stream.download(download_path)
        
        self.download_path = download_path
        self.dance_name = yt.title
        
    def show_dance(self):
        mp_drawing = mp.solutions.drawing_utils
        mp_holistic = mp.solutions.holistic
        # Open Dance Video
        # video_path = input("연습할 춤의 비디오 경로: ")
        dance_video = cv2.VideoCapture("./video/[주간아 직캠] IVE YUJIN - LOVE DIVE (아이브 유진 - 러브 다이브) l EP556.mp4")
        # Open Webcam
        try:
            user_video = cv2.VideoCapture(0)
        except:
            user_video = cv2.VideoCapture(1)
        user_video.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
        user_video.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
        # For Mac & Linux Err
        cv2.startWindowThread()
        with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
            while user_video.isOpened():
                dance_ret, dance_image = dance_video.read()
                user_ret, user_image = user_video.read()
                # 안무 영상 종료시 종료
                if not dance_ret: break
                user_h, user_w = user_image.shape[0], user_image.shape[1]
                
                dance_image = cv2.cvtColor(dance_image, cv2.COLOR_BGR2RGB)
                # user_image = cv2.cvtColor(user_image, cv2.COLOR_BGR2RGB)
                
                dance_image.flags.writeable = False
                # user_image.flags.writeable = False
                dance_results = holistic.process(dance_image)
                
                dance_image = cv2.cvtColor(dance_image, cv2.COLOR_RGB2BGR)
                # user_image = cv2.cvtColor(user_image, cv2.COLOR_RGB2BGR)
                
                # dance_results.pose_landmarks.landmark.x = dance_results.pose_landmarks.landmark.x/user_w*dance_w
                # dance_results.pose_landmarks.landmark.y = dance_results.pose_landmarks.landmark.x/user_h*dance_h
                print(dance_results.pose_landmarks.landmark)
                # Draw Dance
                # mp_drawing.draw_landmarks(user_image, dance_results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                #                         landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 102, 178), thickness=1, circle_radius=1),
                #                         connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 102, 178), thickness=2, circle_radius=1))
                # mp_drawing.draw_landmarks(user_image, dance_results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                #                         landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 102, 178), thickness=1, circle_radius=1),
                #                         connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 102, 178), thickness=2, circle_radius=1))
                mp_drawing.draw_landmarks(user_image, dance_results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                                        landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255, 102, 102), thickness=1, circle_radius=1),
                                        connection_drawing_spec=mp_drawing.DrawingSpec(color=(255, 102, 102), thickness=3, circle_radius=1))
                # for idx, lmk in enumerate(dance_results.pose_landmarks.landmark):
                #     norX, norY = int(lmk.x/user_w*dance_w), int(lmk.y/user_h*dance_h)
                #     cv2.circle(user_image, (norX, norY), 10, (255, 0, 0), cv2.FILLED)
        
                
                h_output = np.hstack((cv2.flip(dance_image, 1), cv2.flip(user_image, 1)))
                cv2.imshow("JustDDance!", h_output)
                if cv2.waitKey(1)==ord("q"): break
                
        dance_video.release()
        user_video.release()
        # For Mac & Linux Err
        cv2.destroyAllWindows()
        cv2.waitKey(1)  